## Day 11: Seating System 

Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can 
finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!

By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make
a quick map of the seat layout (your puzzle input).

The seat layout fits neatly on a grid. Each position is either floor (.), an empty seat (L), or an occupied seat (#). For example, the initial seat 
layout might look like this:
```
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
```
Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All 
decisions are based on the number of occupied seats adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat).
The following rules are applied to every seat simultaneously:

```
If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
Otherwise, the seat's state does not change.
Floor (.) never changes; seats don't move, and nobody sits on the floor.
```

After one round of these rules, every seat in the example layout becomes occupied:
```
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
```
After a second round, the seats with four or more occupied adjacent seats become empty again:
```
#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##
```
This process continues for three more rounds:
```
#.##.L#.##
#L###LL.L#
L.#.#..#..
#L##.##.L#
#.##.LL.LL
#.###L#.##
..#.#.....
#L######L#
#.LL###L.L
#.#L###.##

#.#L.L#.##
#LLL#LL.L#
L.L.L..#..
#LLL.##.L#
#.LL.LL.LL
#.LL#L#.##
..L.L.....
#L#LLLL#L#
#.LLLLLL.L
#.#L#L#.##

#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##
```
At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state!
Once people stop moving around, you count 37 occupied seats.

Simulate your seating area by applying the seating rules repeatedly until no seats change state. **How many seats end up occupied?**

In [ ]:
# Will need later to make deepcopy of layout
import copy
import math

In [ ]:
data = list(map(str, open('../data/day11_input.txt', 'r').read().split('\n')))
data = [list(row) for row in data]

example = list(map(str, open('../examples/day11_example.txt', 'r').read().split('\n')))
example = [list(row) for row in example]

In [ ]:
# Each seat (y,x) will have at most 8 neighbors, let's make a function to return these.
def neighbors(y,x):
    return [(y-1,x-1), (y,x-1), (y+1,x-1), (y-1,x), (y+1,x), (y-1, x+1), (y,x+1), (y+1, x+1)]

This is my first time needing to use deepcopy instead of just normal copy. This is because I need to continue to make copies on top of copies.
See this [StackOverflow post](https://stackoverflow.com/questions/17246693/what-is-the-difference-between-shallow-copy-deepcopy-and-normal-assignment-oper) for more details.

In [ ]:
 def find_occupied_seats(d, part):
    while True:
        new_layout = copy.deepcopy(d)
        columns = len(d[0])
        rows = len(d)
        for y in range(rows):
            for x in range(columns):
                if d[y][x] != ".":  # I want to ignore the floor
                    if part == 1:
                        seats = neighbors(y,x) # Now we want to set up a counter and go through the neighbors.
                    else:
                        seats = visible_seats(y,x, d, rows, columns)
                    count = 0
                    for point in seats: # need to check if point is in the layout, and if its occupied add to count.
                        y_i, x_i = point[0], point[1]
                        if 0 <= y_i <= rows -1 and 0 <= x_i <= columns - 1:
                            if d[y_i][x_i] == "#":
                                count +=1
                    if part == 1:  
                        if d[y][x] == "#" and count >= 4:
                            new_layout[y][x] = "L"
                        elif d[y][x] == "L" and count == 0:
                            new_layout[y][x] = "#"
                    else:
                        if d[y][x] == "#" and count >= 5:
                            new_layout[y][x] = "L"
                        elif d[y][x] == "L" and count == 0:
                            new_layout[y][x] = "#"
        if new_layout == d:
            answer = 0
            for row in new_layout:
                answer += row.count("#")
            return answer
        d = copy.deepcopy(new_layout) # So that we repeat the process on our new grid.


In [ ]:
find_occupied_seats(example,1)

37

In [ ]:
# Answer to part 1
find_occupied_seats(data,1)

2238

## Part Two
As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about the first seat 
they can see in each of those eight directions!

Now, instead of considering just the eight immediately adjacent seats, consider the first seat in each of those eight directions. For example, the 
empty seat below would see eight occupied seats:
```
.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
```
The leftmost empty seat below would only see one empty seat, but cannot see any of the occupied ones:
```
.............
.L.L.#.#.#.#.
.............
```
The empty seat below would see no occupied seats:
```
.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
```
Also, people seem to be more tolerant than you expected: it now takes five or more visible occupied seats for an occupied seat to become empty 
(rather than four or more from the previous rules). The other rules still apply: empty seats that see no occupied seats become occupied, seats 
matching no rule don't change, and floor never changes.

Given the same starting layout as above, these new rules cause the seating area to shift around as follows:
```
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL

#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##

#.LL.LL.L#
#LLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLLL.L
#.LLLLL.L#

#.L#.##.L#
#L#####.LL
L.#.#..#..
##L#.##.##
#.##.#L.##
#.#####.#L
..#.#.....
LLL####LL#
#.L#####.L
#.L####.L#

#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##LL.LL.L#
L.LL.LL.L#
#.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLL#.L
#.L#LL#.L#

#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.#L.L#
#.L####.LL
..#.#.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#

#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
```
Again, at this point, people stop shifting around and the seating area reaches equilibrium. Once this occurs, you count 26 occupied seats.

Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, **how many seats end up occupied?**

This is just like last year's [Day 10 asteroid's problem](https://adventofcode.com/2019/day/10)! I want to solve it in a similar way, 
by using atan2 to calculate theangles at the points and keep the right and 45-degree angles.

In [ ]:
def visible_seats(y,x,d,rows,columns):
    visible_seats = []
    neg_threepi4_dict, neg_pi2_dict, neg_pi4_dict, zero_dict, pi4_dict, pi2_dict, threepi4_dict, pi_dict = {}, {}, {}, {}, {}, {}, {}, {}
    dicts = [neg_threepi4_dict, neg_pi2_dict, neg_pi4_dict, zero_dict, pi4_dict, pi2_dict, threepi4_dict, pi_dict]
    for j in range(rows):
        for i in range(columns):
            if i == x and j == y:
                continue
            else:
                if (atan2(y-j, x-i) == -3*math.pi/4) and (d[j][i]== "#" or d[j][i]=="L"):
                    neg_threepi4_dict[(j, i)] = math.sqrt((x-i)**2+(y-j)**2) 
                elif atan2(y-j, x-i) == -math.pi/2 and (d[j][i]== "#" or d[j][i]=="L"):
                    neg_pi2_dict[(j, i)] = math.sqrt((x-i)**2+(y-j)**2) 
                elif atan2(y-j, x-i) == -math.pi/4 and (d[j][i]== "#" or d[j][i]=="L"):
                    neg_pi4_dict[(j, i)] = math.sqrt((x-i)**2+(y-j)**2) 
                elif atan2(y-j, x-i) == 0 and (d[j][i]== "#" or d[j][i]=="L"):
                    zero_dict[(j, i)] = math.sqrt((x-i)**2+(y-j)**2) 
                elif atan2(y-j, x-i) ==  math.pi/4 and (d[j][i]== "#" or d[j][i]=="L"):
                    pi4_dict[(j, i)] = math.sqrt((x-i)**2+(y-j)**2) 
                elif atan2(y-j, x-i) == math.pi/2 and (d[j][i]== "#" or d[j][i]=="L"):
                    pi2_dict[(j, i)] = math.sqrt((x-i)**2+(y-j)**2) 
                elif atan2(y-j, x-i) == 3*math.pi/4 and (d[j][i]== "#" or d[j][i]=="L"):
                    threepi4_dict[(j, i)] = math.sqrt((x-i)**2+(y-j)**2) 
                elif atan2(y-j, x-i) == math.pi and (d[j][i]== "#" or d[j][i]== "L"):
                    pi_dict[(j, i)] = math.sqrt((x-i)**2+(y-j)**2)
    for di in dicts:
        if len(di)!= 0:
            visible_seats.append(min(di, key=di.get))
    return visible_seats


In [ ]:
find_occupied_seats(example, 2)

26

In [ ]:
#Part two answer (takes a long time to run)
find_occupied_seats(data,2)

2013

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=320b6760-c2f1-4eb8-a5b7-b5aca5d8cf36' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>